In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd    

from models.aa_common.data_loader import get_pmd_dataset, get_patho_and_likelypatho_SNVs, get_population_freq_SNVs

In [29]:
patho_variants_df = get_patho_and_likelypatho_SNVs(home_dir)
print("#-of rows mapped to rsids: ", patho_variants_df[~pd.isna(patho_variants_df["snp_id"])].shape[0])
patho_rs_ids_set = set(patho_variants_df[~pd.isna(patho_variants_df["snp_id"])]["snp_id"].values)
len(patho_rs_ids_set)

raw data: (10282, 13)
Index(['clinvar_id', 'gene_symbol', 'gene_id', 'snp_id', 'chrom_acc_version',
       'chrom_pos', 'ref_allele', 'alt_allele', 'prot_acc_version', 'prot_pos',
       'wt', 'mut', 'class'],
      dtype='object')

Log: excluding variants corresponding to proteins having seq-len>1022 ...

Log: Loading combined fasta iterator ...
#-protein sequences (seq-len<=1022): 1434
#-of rs-ids mapped to pathogenicity dataset:  331
Pathogenic: 2308, Likely pathogenic: 4168, total: (6476, 13)
#-of rows mapped to rsids:  331


329

In [30]:
pmd_variants_df = get_pmd_dataset(home_dir)
print("#-of rows mapped to rsids: ", pmd_variants_df[~pd.isna(pmd_variants_df["snp_id"])].shape[0])
pmd_rs_ids_set = set(pmd_variants_df[~pd.isna(pmd_variants_df["snp_id"])]["snp_id"].values)
len(pmd_rs_ids_set)


Log: Loading Protein Mutation Dataset (PMD) ...
(8582, 30)
Index(['mut_id', 'md5', 'pmd_id', 'nr', 'mut_real', 'chrom', 'chrom_pos',
       'ref_allele', 'alt_allele', 'function', 'source', 'crossref', 'snp_id',
       'CADD_raw', 'MetaRNN_score', 'MVP_score', 'Polyphen2_HVAR_score',
       'REVEL_score', 'SIFT_score', 'integrated_fitCons_score',
       'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic', 'seq',
       'wt', 'mut', 'prot_pos', 'function_summarized', 'functional_effect',
       'pmd_nr_id'],
      dtype='object')

Log: excluding variants corresponding to proteins having seq-len>1022 ...

Log: Loading combined fasta iterator ...
#-protein sequences (seq-len<=1022): 2198
(8582, 30)
Effect       4633
Knock-out    1981
No-effect    1968
Name: functional_effect, dtype: int64
#-of rows mapped to rsids:  3583


1768

In [7]:
rs_ids_set = pmd_rs_ids_set.union(patho_rs_ids_set)
print(len(rs_ids_set))

2088


In [10]:
popu_variants_df = get_population_freq_SNVs(home_dir)
popu_variants_df = popu_variants_df[popu_variants_df["class"]!="Zero-population"]


Log: Loading data ...
(2865836, 14)
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq', 'class'],
      dtype='object')
Zero-population    2042590
Singleton           462444
Ultra-rare          314013
Rare                 28622
Common               18167
Name: class, dtype: int64


In [20]:
overlapping_df = popu_variants_df[popu_variants_df["snp_id"].isin(rs_ids_set)] # popu_variants_overlapped_in_pmd_and_patho
print(overlapping_df.shape)
overlapping_df["class"].value_counts()

(492, 14)


Ultra-rare    214
Singleton     170
Common         64
Rare           44
Name: class, dtype: int64

In [25]:
non_overlapping_df = popu_variants_df[~popu_variants_df["snp_id"].isin(rs_ids_set)]

common = non_overlapping_df[non_overlapping_df["class"]=="Common"].sample(10000)
rare = non_overlapping_df[non_overlapping_df["class"]=="Rare"].sample(10000)
ultra_rare = non_overlapping_df[non_overlapping_df["class"]=="Ultra-rare"].sample(10000)
singleton = non_overlapping_df[non_overlapping_df["class"]=="Singleton"].sample(10000)
sampled_popu_variants_df = pd.concat([overlapping_df, common, rare, ultra_rare, singleton])
sampled_popu_variants_df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
sampled_popu_variants_df["class"].value_counts()

Ultra-rare    10214
Singleton     10170
Common        10064
Rare          10044
Name: class, dtype: int64